### Imports

In [13]:
# imports 
import pandas as pd
import warnings
import pickle
warnings.filterwarnings('ignore')

In [14]:
# defs
def remove_emoji(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')
def generate_experiment2(train_size,test_size,added_n,experiments_index):
    #---# A: Kaggle 
    df_train = pd.concat([
        df_kaggle[df_kaggle['type'] != 'ENFJ'][::2],
        df_kaggle[df_kaggle['type'] == 'ENFJ'][:train_size],
        df_kaggle[df_kaggle['type'] == 'ENFJ'][train_size:(train_size+added_n)],
    ]).reset_index(drop=True)
    df_test = pd.concat([
        df_kaggle[df_kaggle['type'] == 'ENFJ'][-test_size:],
        df_kaggle[df_kaggle['type'] != 'ENFJ'][1::2]
    ])
    df_trains_dct[f'{experiments_index}a'] = df_train 
    df_tests_dct[f'{experiments_index}a'] = df_test
    _df1 = df_train[:-added_n]['type'].value_counts().reset_index().assign(Source='Kaggle')
    _df2 = df_train[-added_n:]['type'].value_counts().reset_index().assign(Source='Kaggle')
    _train = pd.concat([_df1,_df2],axis=0).reset_index(drop=True).assign(DataType = 'Train')
    _test = df_test['type'].value_counts().reset_index().assign(DataType = 'Test',Source='Kaggle')
    tidydata1= pd.concat([_train,_test]).reset_index(drop=True).assign(col = lambda df: df.Source.map({'Kaggle':0,'ChatGPT':1})).query('type == "ENFJ"')
    #---# B: GPT
    df_train = pd.concat([
        df_kaggle[df_kaggle['type'] != 'ENFJ'][::2],
        df_kaggle[df_kaggle['type'] == 'ENFJ'][:train_size],
        df_gpt_ENFJ[df_gpt_ENFJ['type'] == 'ENFJ'][:added_n],
    ]).reset_index(drop=True)
    df_test = pd.concat([
        df_kaggle[df_kaggle['type'] == 'ENFJ'][-test_size:],
        df_kaggle[df_kaggle['type'] != 'ENFJ'][1::2]
    ])
    df_trains_dct[f'{experiments_index}b'] = df_train 
    df_tests_dct[f'{experiments_index}b'] = df_test
    _df1 = df_train[:-added_n]['type'].value_counts().reset_index().assign(Source='Kaggle')
    _df2 = df_train[-added_n:]['type'].value_counts().reset_index().assign(Source='ChatGPT')
    _train = pd.concat([_df1,_df2],axis=0).reset_index(drop=True).assign(DataType = 'Train')
    _test = df_test['type'].value_counts().reset_index().assign(DataType = 'Test',Source='Kaggle')
    tidydata2 = pd.concat([_train,_test]).reset_index(drop=True).assign(col = lambda df: df.Source.map({'Kaggle':0,'ChatGPT':1})).query('type=="ENFJ"')
    #---# C: Kaggle+GPT
    df_train = pd.concat([
        df_kaggle[df_kaggle['type'] != 'ENFJ'][::2],        
        df_kaggle[df_kaggle['type'] == 'ENFJ'][:train_size],
        df_gpt_ENFJ[df_gpt_ENFJ['type'] == 'ENFJ'][:added_n],
        df_kaggle[df_kaggle['type'] == 'ENFJ'][train_size:(train_size+added_n)],
    ]).reset_index(drop=True)
    df_test = pd.concat([
        df_kaggle[df_kaggle['type'] == 'ENFJ'][-test_size:],
        df_kaggle[df_kaggle['type'] != 'ENFJ'][1::2]
    ])
    df_trains_dct[f'{experiments_index}c'] = df_train 
    df_tests_dct[f'{experiments_index}c'] = df_test
    _df1 = df_train[:-added_n]['type'].value_counts().reset_index().assign(Source='Kaggle')
    _df2 = df_train[-added_n:]['type'].value_counts().reset_index().assign(Source='ChatGPT')
    _train = pd.concat([_df1,_df2],axis=0).reset_index(drop=True).assign(DataType = 'Train')
    _test = df_test['type'].value_counts().reset_index().assign(DataType = 'Test',Source='Kaggle')
    tidydata3 = pd.concat([_train,_test]).reset_index(drop=True).assign(col = lambda df: df.Source.map({'Kaggle':0,'ChatGPT':1})).query('type=="ENFJ"')
    #---#
    tidydata = pd.concat([tidydata1.assign(Setting = 'Real'),tidydata2.assign(Setting = 'Synthetic'),tidydata2.assign(Setting = 'Real+Synthetic')])
    tidydata.to_csv(f"tidy_csv/tidydata_{experiments_index.replace('/','')}.csv",index=False)    

def generate_experiment3(train_size,test_size,added_n,experiments_index):
    #---# A: Kaggle
    df_train = pd.concat([
        df_kaggle.query('type in ["ISTP","ESTP","INTP","ISFP","ISTJ"]')[::2],
        df_kaggle[df_kaggle['type'] == 'ENFJ'][:train_size],
        df_kaggle[df_kaggle['type'] == 'ENFJ'][train_size:(train_size+added_n)],
    ]).reset_index(drop=True)
    df_test = pd.concat([
        df_kaggle.query('type in ["ISTP","ESTP","INTP","ISFP","ISTJ"]')[::2],
        df_kaggle[df_kaggle['type'] == 'ENFJ'][-test_size:],        
    ])
    df_trains_dct[f'{experiments_index}a'] = df_train 
    df_tests_dct[f'{experiments_index}a'] = df_test
    _df1 = df_train[:-added_n]['type'].value_counts().reset_index().assign(Source='Kaggle')
    _df2 = df_train[-added_n:]['type'].value_counts().reset_index().assign(Source='Kaggle')
    _train = pd.concat([_df1,_df2],axis=0).reset_index(drop=True).assign(DataType = 'Train')
    _test = df_test['type'].value_counts().reset_index().assign(DataType = 'Test',Source='Kaggle')
    tidydata1= pd.concat([_train,_test]).reset_index(drop=True).assign(col = lambda df: df.Source.map({'Kaggle':0,'ChatGPT':1}))
    #---# B: GPT
    df_train = pd.concat([
        df_kaggle.query('type in ["ISTP","ESTP","INTP","ISFP","ISTJ"]')[::2],
        df_kaggle[df_kaggle['type'] == 'ENFJ'][:train_size],
        df_gpt[df_gpt['type'] == 'ENFJ'][:added_n],
    ]).reset_index(drop=True)
    df_test = pd.concat([
        df_kaggle[df_kaggle['type'] == 'ENFJ'][-test_size:],
        df_kaggle.query('type in ["ISTP","ESTP","INTP","ISFP","ISTJ"]')[::2],
    ])
    df_trains_dct[f'{experiments_index}b'] = df_train 
    df_tests_dct[f'{experiments_index}b'] = df_test
    _df1 = df_train[:-added_n]['type'].value_counts().reset_index().assign(Source='Kaggle')
    _df2 = df_train[-added_n:]['type'].value_counts().reset_index().assign(Source='ChatGPT')
    _train = pd.concat([_df1,_df2],axis=0).reset_index(drop=True).assign(DataType = 'Train')
    _test = df_test['type'].value_counts().reset_index().assign(DataType = 'Test',Source='Kaggle')
    tidydata2 = pd.concat([_train,_test]).reset_index(drop=True).assign(col = lambda df: df.Source.map({'Kaggle':0,'ChatGPT':1}))
    tidydata = pd.concat([tidydata1.assign(Setting = 'Real'),tidydata2.assign(Setting = 'Synthetic')])
    tidydata.to_csv(f"tidy_csv/tidydata_{experiments_index.replace('/','')}.csv",index=False)    

### data

In [15]:
df_kaggle = pd.read_csv("data/df_kaggle.csv")
df_gpt = pd.read_csv("data/df_gpt.csv")
df_gpt_ENFJ = df_gpt[df_gpt['type'] == 'ENFJ'][:160] 
df_gpt_ESTP = df_gpt[df_gpt['type'] == 'ESTP'][:155]
df_gpt_ESFP = df_gpt[df_gpt['type'] == 'ESFP'][:176]
df_gpt_ESFJ = df_gpt[df_gpt['type'] == 'ESFJ'][:179]
df_gpt_ESTJ = df_gpt[df_gpt['type'] == 'ESTJ'][:181]
df_trains_dct = dict()
df_tests_dct = dict()

### Experiment 1: Setup

In [16]:
#!mkdir tidy_csv

*scenario1*

In [17]:
df_train = df_kaggle[::2].reset_index(drop=True)
df_test = df_kaggle[1::2].reset_index(drop=True)
df_trains_dct['experiment1/scenario1'] = df_train 
df_tests_dct['experiment1/scenario1'] = df_test 
_train = df_train['type'].value_counts().reset_index().assign(DataType = 'Train')
_test = df_test['type'].value_counts().reset_index().assign(DataType = 'Test')
tidydata = pd.concat([_train,_test]).reset_index(drop=True)
tidydata.to_csv("tidy_csv/tidydata_experiment1scenario1.csv",index=False)

*scenario2*

In [18]:
df_train = pd.concat([
    df_kaggle[::2],
    df_gpt_ESTP,
    df_gpt_ESFP,
    df_gpt_ESFJ,
    df_gpt_ESTJ,
]).reset_index(drop=True)
df_test = df_kaggle[1::2].reset_index(drop=True)
df_trains_dct['experiment1/scenario2'] = df_train 
df_tests_dct['experiment1/scenario2'] = df_test 
n = len(df_kaggle[::2])
_df1 = df_train[:n]['type'].value_counts().reset_index().assign(Source='Kaggle')
_df2 = df_train[n:]['type'].value_counts().reset_index().assign(Source='ChatGpt')
_train = pd.concat([_df1,_df2],axis=0).reset_index(drop=True).assign(DataType = 'Train')
_test = df_test['type'].value_counts().reset_index().assign(DataType = 'Test',Source='Kaggle')
tidydata = pd.concat([_train,_test]).reset_index(drop=True).assign(col = lambda df: df.Source.map({'Kaggle':0,'ChatGPT':1}))
tidydata.to_csv("tidy_csv/tidydata_experiment1scenario2.csv",index=False)

### Experiment 2: Setup

In [19]:
generate_experiment2(train_size=1,test_size=30,added_n=0,experiments_index='experiment2/scenario0')
generate_experiment2(train_size=1,test_size=30,added_n=40,experiments_index='experiment2/scenario1')
generate_experiment2(train_size=1,test_size=30,added_n=80,experiments_index='experiment2/scenario2')
generate_experiment2(train_size=1,test_size=30,added_n=120,experiments_index='experiment2/scenario3')
generate_experiment2(train_size=1,test_size=30,added_n=159,experiments_index='experiment2/scenario4')

### Experiment 3: Setup

In [20]:
generate_experiment3(train_size=1,test_size=40,added_n=0,experiments_index='experiment3/scenario0')
generate_experiment3(train_size=1,test_size=40,added_n=20,experiments_index='experiment3/scenario1')
generate_experiment3(train_size=1,test_size=40,added_n=40,experiments_index='experiment3/scenario2')
generate_experiment3(train_size=1,test_size=40,added_n=60,experiments_index='experiment3/scenario3')
generate_experiment3(train_size=1,test_size=40,added_n=80,experiments_index='experiment3/scenario4')
generate_experiment3(train_size=1,test_size=40,added_n=100,experiments_index='experiment3/scenario5')
generate_experiment3(train_size=1,test_size=40,added_n=120,experiments_index='experiment3/scenario6')
generate_experiment3(train_size=1,test_size=40,added_n=140,experiments_index='experiment3/scenario7')

### Save experimental setup

In [21]:
!rm -rf experiment_setup
![ -d "experiment_setup" ] || mkdir "experiment_setup"
with open(file='experiment_setup/df_trains_dct.pickle', mode='wb') as f:
    pickle.dump(obj=df_trains_dct,file=f)    
with open(file='experiment_setup/df_tests_dct.pickle', mode='wb') as f:
    pickle.dump(obj=df_tests_dct,file=f)        